In [ ]:
import pandas as pd
from collections import Counter
import tkinter as tk
import pylab as plt
import networkx as nx   
from datetime import datetime
import duration

columnss = ['A_Accepted', 'A_Complete', 'A_Concept', 'A_Create Application',
       'A_Incomplete', 'A_Pending', 'A_Submitted', 'A_Validating',
       'O_Accepted', 'O_Cancelled', 'O_Create Offer', 'O_Created',
       'O_Returned', 'O_Sent (mail and online)', 'O_Sent (online only)',
       'W_Assess potential fraud', 'W_Call after offers',
       'W_Call incomplete files', 'W_Complete application', 'W_Handle leads',
       'W_Shortened completion', 'W_Validate application']

users_index = ['User_'+str(i) for i in range(1,144)]
file ='D:\\Apending_time.csv'
activity = 'W_Validate application'
exe_weight = 0.3
trans_weight = 0.1
time_weight =0.6



def importfiles(f):
    f1 = open(f,'r')
    data =[]
    
    for line in f1:
        split_line=line.split(',')
        data.append(split_line)
    
    f1.close()
    headers = data[0]
    
    data = data[1:len(data)]
    apending = pd.DataFrame(data,columns =headers)

    return apending

#--------------------------------------------------------------------

def M_actvity(rows,columns):   
    
    apending = importfiles(file)
    
    activity_user = apending.groupby('Activity')['Resource'].apply(list)
    
    #index -----------------------------------------------------------------------------

    
    column = [i for i in range(len(columns))]
    
    
#activity matrix-----------------------------------------------------------------------------------
    
    matrix_activity = pd.DataFrame(index = rows, columns = column)
    
    a = list()
    for i in range(len(activity_user)):
        a.append([[x,(activity_user.values[i]).count(x)] for x in set(activity_user.values[i])])
    
    for k in range(len(a)):
        for i in range(len(matrix_activity)):
            for j in range(len(a[k])):
                if matrix_activity.index[i] == a[k][j][0]:
                    matrix_activity.loc[a[k][j][0],k] =a[k][j][1]
    
    matrix_activity = matrix_activity.fillna(0)
    matrix_activity.columns = columns
    
    
    matrix_activity = matrix_activity[activity]
    matrix_activity = matrix_activity[matrix_activity>0]
    
    return matrix_activity
    
#handover matrix---------------------------------------------------------------------
    
def M_handover(rows): 
    
    apending = importfiles(file)
    
    matrix_handover = pd.DataFrame(index = rows, columns = rows)
    
    activities = apending[apending['Activity'] == activity]
    pendingindex = activities.index
    pendingindex = list(pendingindex)
    
    newindex = getbeforeafter(pendingindex)
    
    new = list()
    for i in range(len(newindex)):
        new.append(apending[apending.index == newindex[i]])
    
    new_pd = pd.DataFrame(new[0])
    
    for i in range(1,len(new)):
        new_pd = new_pd.append(new[i])
           
    handover_user = apending.groupby('Case ID')['Resource'].apply(list)
    handover_user = list(handover_user)
    
    
    hand = list()
    for i in range(len(handover_user)):
        for j in range(len(handover_user[i])-1):
            hand.append((handover_user[i][j],handover_user[i][j+1]))    
    
    aca = Counter(hand)
    
    usertouser = aca.keys()
    usertouser = list(usertouser)
    
    usertofreq = aca.values()
    usertofreq = list(usertofreq)
    
    for i in range(len(matrix_handover)):
        for j in range(len(matrix_handover)):    
            for k in range(len(usertouser)):
                if (matrix_handover.index[i] == usertouser[k][0]) and (matrix_handover.columns[j] == usertouser[k][1]):
                    matrix_handover.loc[usertouser[k][0],usertouser[k][1]] = usertofreq[k]
                    
                    
    matrix_handover = matrix_handover.fillna(0)

    return matrix_handover

 #--------------------------------------------------------------------
def getbeforeafter(a):
    newindex=list()
    for i in range(len(a)):
        x=0
        y=0
        if a[i] == 0:
            return
        else:
            x = a[i]-1
            y = a[i]+1
        newindex.append(x)
        newindex.append(a[i])
        newindex.append(y)

    return newindex    

#--------------------------------------------------------------------
    
def doscalculator(a,b):
    x=0.0
    y=0.0
    for i in range(len(a)):
        if a[i] > b[i]:
            x+= b[i]/a[i]
        elif a[i] < b[i]: 
            x+=1
        if a[i] > 0:
            y+=1
        else:
            y+=0
            
    return x/y   

#--------------------------------------------------------------------

def D_task():
    
    matrix_activity = M_actvity(users_index,columnss)
    DOS_task = pd.DataFrame(index = users_index, columns = users_index)
    
    for i in range(len(DOS_task)):
        for j in range(len(DOS_task)):
            if i ==j:
                DOS_task.loc[DOS_task.index[i],DOS_task.columns[j]] = 1
            if (DOS_task.index[i] in matrix_activity.index) and (DOS_task.columns[j] in matrix_activity.index):
                DOS_task.loc[DOS_task.index[i],DOS_task.columns[j]] = matrix_activity.loc[DOS_task.columns[j]] / matrix_activity.loc[DOS_task.index[i]]
            else:
                DOS_task.loc[DOS_task.index[i],DOS_task.columns[j]] =0        
                            
    return DOS_task
        
#--------------------------------------------------------------------

def D_transfer():
    
    matrix_handover = M_handover(users_index)
    DOS_transfer = pd.DataFrame(index = users_index, columns = users_index)
    
    for i in range(len(DOS_transfer)):
        for j in range(len(DOS_transfer)):
            DOS_transfer.loc[DOS_transfer.index[i],DOS_transfer.columns[j]] = doscalculator(list(matrix_handover.loc[DOS_transfer.index[i],]),list(matrix_handover.loc[DOS_transfer.columns[j]]))
            if i==j:
                DOS_transfer.loc[DOS_transfer.index[i],DOS_transfer.columns[j]] = 1
    
    return DOS_transfer    
    

# the length of time -------------------------------------------------------------

def timeall():
    
    timediff = list()
    
    apending = importfiles(file)
    activities = apending[apending['Activity'] == activity]
    activities = activities.reset_index()
    
    for i in range(len(activities)):
        time =datetime.strptime(activities['Start Timestamp'][i],'%Y-%m-%d %H:%M') -datetime.strptime(activities['Complete Timestamp'][i],'%Y-%m-%d %H:%M')
        timediff.append(time)
    
    timediff = list(timediff)
    
    timeinsecond= list()
    for i in range(len(timediff)):
        timeinsecond.append(duration.to_seconds(timediff[i]))
    
    timedf = pd.DataFrame(index= activities['Resource'])
    timedf['timediff'] = timeinsecond
    
    timedf = timedf.groupby(timedf.index)['timediff'].sum()
    timedf = timedf[timedf >0]      
    
    time_total = pd.DataFrame(index = users_index, columns = users_index)
    
    
    for i in range(len(time_total)):
        for j in range(len(time_total)):
            if (time_total.index[i] in timedf.index) and (time_total.columns[j] in timedf.index):
                if (timedf.loc[time_total.index[i]] != 0) and (timedf.loc[time_total.index[j]] != 0): #¼ýÀÚ/¼ýÀÚ                 
                    time_total.loc[time_total.index[i],time_total.columns[j]] = timedf.loc[time_total.columns[j]] / timedf.loc[time_total.index[i]]                                                   
                if i == j:
                    time_total.loc[time_total.index[i],time_total.columns[j]] = 1
            else:
                time_total.loc[time_total.index[i],time_total.columns[j]] = 0
    
    for i in range(len(time_total)):
        for j in range(len(time_total)):
            if time_total.loc[time_total.index[i]][j] !=0:
                time_total.loc[time_total.index[i]][j] = 1.0/time_total.loc[time_total.index[i]][j]
                
    return time_total

# weighting between experience and time ----------------------------------------------------
    
def weighting(exe, trans,time):
    Exe_weight = exe
    Trans_weight = trans
    Time_weight = time
    
    DOS_Time = pd.DataFrame(index = users_index, columns = users_index)
    
    dos_task = D_task()
    dos_transfer = D_transfer()
    Time_total = timeall()
    
    for i in range(len(DOS_Time)):
        for j in range(len(DOS_Time)):
            DOS_Time.loc[DOS_Time.index[i]][j]  = dos_task.loc[dos_task.index[i]][j]*Exe_weight + dos_transfer.loc[dos_transfer.index[i]][j]*Trans_weight + Time_total.loc[Time_total.index[i]][j]*Time_weight
    
    return DOS_Time


#get the best resource ------------------------------------------------------------
    

def showing():
    DOS_Time = weighting(exe_weight,trans_weight,time_weight)
    realmax = list()            
    for i in range(len(DOS_Time)):
        values = set(list(DOS_Time.loc[DOS_Time.index[i]]))
        values.discard(DOS_Time.loc[DOS_Time.index[i]][i])
        realmax.append(max(values))            
    
    indices = list()
    for i in range(len(realmax)):
        indices.append(list(DOS_Time.loc[DOS_Time.index[i]]).index(realmax[i]))   
        
    bestresourceindex = [str(i+1) for i in indices]
    
        
    Bestresource = pd.DataFrame(index = users_index)
        
    Bestresource['Replace resource'] = bestresourceindex
    Bestresource['Replace score'] = realmax


    class SampleApp(tk.Tk):
        def __init__(self, *args, **kwargs):
            tk.Tk.__init__(self, *args, **kwargs)
            lb = tk.Listbox(self)
            for i in range(len(DOS_Time.index)):
                lb.insert("end",DOS_Time.index[i])
            lb.bind("<Double-Button-1>", self.OnDouble)
            lb.pack(side="top", fill="both", expand=True)
    
        def OnDouble(self, event):
            widget = event.widget
            selection=widget.curselection()
            value = widget.get(selection[0])
            print("selection:", selection, ": '%s'" % value)
            g = nx.Graph()
            for i in range(len(DOS_Time.columns)):
                g.add_edge(value,DOS_Time.columns[i], weight = DOS_Time[value][i])
            nx.draw_networkx(g)
            plt.show()
            print(DOS_Time.loc[value])
            print("Best resource is" ,Bestresource.loc[value][0], "and its score is" ,Bestresource.loc[value][1])
    
    if __name__ == "__main__":
        app = SampleApp()
        app.mainloop()
        
    return Bestresource

#-----------------------------------------------------------------
showing()

